So, I can get all the game titles and then figure out the url request to get it's individual page, or I can try to scrape the url directly.  I can use the same name scraping to get users

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import requests
import csv
import time

In [2]:
console_name_list = ['xboxone', 'ps4', 'wii-u', 'pc', 'switch', '3ds', 'ios', 'vita']

 * Get request
 * Get html from source using BeautifulSoup

# Creating a logging function for debugging

In [3]:
def log_init():
    with open('log.txt', 'w') as log:
            log.write('Beginning Log\n')
def log_write(text):
    with open('log.txt', 'a') as log:
            log.write(f'{text}\n')

# First, Run a function that extracts games from the list of games by console

In [4]:
def scrape_list(soup):
    
    ol_of_games = soup.find_all(name="li", attrs={'class': "game_product"})

    game_dicts = []
    for entry in ol_of_games:

        game = {}
        link = entry.find("a")
        critic_score = entry.find("div", attrs={'class': "metascore_w"}).text
        user_score = entry.find("span", attrs={'class' : 'textscore'}).text
        game['title'] = link.text.strip()
        game['page'] = link.attrs['href']
        game['critic_score'] = critic_score
        game['user_score'] = user_score
        game_dicts.append(game)
    return game_dicts

In [5]:
def scrape_console(console):
    log_init()
    game_url = f"http://www.metacritic.com/browse/games/release-date/available/{console}/metascore?page=0"
    # Using requests now
    headers = {'User-agent': 'no this is patrick .1'}
    res = requests.get(game_url, headers=headers)
    if res.status_code != 200:
        print("Couldn't reach page")
        return
    
    # go to the metacritic game page
    page_source = res.content
    # Get HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    
    ## Count number of pages 
    
    try: # Idk if the try exept works.  Do I use catch here?
        pagelist = soup.find("ul", attrs={'class': "pages"})
        num_pages = len(pagelist.find_all("li"))
    except:
        print("couldn't reach url")
        return

    cur_page = 0
    game_dicts = []
    while(cur_page < num_pages):
        game_dicts.extend(scrape_list(soup))
        cur_page +=1
        game_url = f"http://www.metacritic.com/browse/games/release-date/available/{console}/metascore?page={cur_page}"
        log_write(f'scraping {game_url}')
        headers = {'User-agent': f'NOT IGN {cur_page}'}
        res = requests.get(game_url, headers=headers)
        if res.status_code != 200:
        
            log_write(f"Couldn't reach page {cur_page}")
            log_write(f'status_code: {res.status_code}')
            print(f"Couldn't reach page {cur_page}")
            return game_dicts
        page_source = res.content
        soup = BeautifulSoup(page_source, 'html.parser')

        
    return game_dicts
    
    

In [6]:
ps4 = scrape_console("ps4")
ps4[0:10]

[{'critic_score': '97',
  'page': '/game/playstation-4/grand-theft-auto-v',
  'title': 'Grand Theft Auto V',
  'user_score': '8.3'},
 {'critic_score': '95',
  'page': '/game/playstation-4/the-last-of-us-remastered',
  'title': 'The Last of Us Remastered',
  'user_score': '9.1'},
 {'critic_score': '94',
  'page': '/game/playstation-4/god-of-war',
  'title': 'God of War',
  'user_score': '9.2'},
 {'critic_score': '94',
  'page': '/game/playstation-4/xcom-2-war-of-the-chosen',
  'title': 'XCOM 2: War of the Chosen',
  'user_score': '6.2'},
 {'critic_score': '93',
  'page': '/game/playstation-4/persona-5',
  'title': 'Persona 5',
  'user_score': '9.1'},
 {'critic_score': '93',
  'page': '/game/playstation-4/metal-gear-solid-v-the-phantom-pain',
  'title': 'Metal Gear Solid V: The Phantom Pain',
  'user_score': '8.2'},
 {'critic_score': '93',
  'page': '/game/playstation-4/uncharted-4-a-thiefs-end',
  'title': "Uncharted 4: A Thief's End",
  'user_score': '8.3'},
 {'critic_score': '92',
  '

In [7]:
len(ps4)

1588

In [8]:
# adding an index field
for i in range(len(ps4)):
    ps4[i]['GID'] = i

In [9]:
# save list of games 
fields = ps4[0].keys()

with open('ps4.csv', 'w') as g:
    dw = csv.DictWriter(g, fieldnames=fields)
    dw.writeheader()
    dw.writerows(ps4)

# Next, Get data about each game by scraping their individual page

In [10]:
def extract_data_from_console(console):
    log_init()
    log_write('Start Extracting metadata from console')
    start_time = time.time()
    game_list = []
    for game in console:
        new_game_data = extract_data_from_game(game)
        game_list.append(new_game_data)
        log_write(f'Runtime: {time.time()-start_time} ')
    return game_list


def extract_data_from_game(game):
    log_write(f'Extracting data from {game["title"]}')
    #generate url from game object
    url = "http://www.metacritic.com" + game['page'] + "/details"
    
    # get page request and soup object
    headers = {'User-agent': f'1.{game["title"]}'}
    res = requests.get(url, headers=headers)
    
    log_write(f"Accessing page at {url} with status_code {res.status_code}")
    
    if(res.status_code > 300):
        log_write()
        return game
    
    page_source = res.content
    soup = BeautifulSoup(page_source, 'html.parser')
    #Get meta data
    return scrape_meta_data(soup)
    
def scrape_meta_data(soup):
    meta = {}
    soup.find()
    summary_soup = soup.find('div', attrs={'class': 'product_summary'})
    product_soup = soup.find('table', attrs={'cellspacing':'0'})

    try:
        meta['publisher'] = soup.find('li', attrs={'class': 'publisher'}).find('a').text.strip()
    except:
        meta['publisher'] = None
    try:
        meta['release_date'] = soup.find('li', attrs={'class': 'release_data'}).find('span', attrs={'class' : 'data'}).text
    except:
        meta['release_date'] = None
    try:
        meta['summary'] = summary_soup.find('span', attrs={'class':'data'}).text
    except:
        meta['summary'] = None
    try:
        meta['rating'] = product_soup.find(text='Rating:').parent.parent.find('td').text
    except:
        meta['rating'] = None
    try:
        meta['developer'] = product_soup.find(text='Developer:').parent.parent.find('td').text
    except:
        meta['developer'] = None
    try:
        meta['genres'] = product_soup.find(text='Genre(s):').parent.parent.find('td').text.strip().replace("  ", "")
    except:
        meta['genres'] = None
    try:
        meta['online'] = (product_soup.find(text='Number of Online Players:').parent.parent.find('td').text == 'No Online Multiplayer')*1
    except:
        meta['online'] = None
    try:
        meta['num_credits'] = len(soup.find('table', attrs={'class':'credits'}).find_all('tr'))-1
    except:
        meta['num_credits'] = None
    return meta

In [11]:
new_ps4 = extract_data_from_console(ps4)

for g1, g2 in zip(ps4, new_ps4):
    g1.update(g2)

ps4[0:20]

fields = ps4[0].keys()

with open('ps4.csv', 'w') as g:
    dw = csv.DictWriter(g, fieldnames=fields)
    dw.writeheader()
    dw.writerows(ps4)

KeyboardInterrupt: 